In [6]:
import folium
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
print(folium.__version__)
world_map = folium.Map()
world_map

0.12.1.post1


In [9]:
# define the national map
national_map = folium.Map(location=[35.3, 100.6], zoom_start=4)
# display national map
national_map

In [17]:
# define the city map,tiles='Stamen Toner'
city_map = folium.Map(location=[39.93, 116.40], zoom_start=11, tiles='Stamen Toner')
# display city map
city_map


In [32]:
# generated data
import numpy as np
data = (
    np.random.normal(size=(100, 3)) *
    np.array([[0.1, 0.1, 0.1]]) +
    np.array([[40, 116.5, 1]])
).tolist()
data[:3]

from folium.plugins import HeatMap
m = folium.Map([39.93, 116.38], tiles='stamentoner', zoom_start=10)
HeatMap(data).add_to(m)

folium.Circle(
    radius=200,
    location=[39.92, 116.43],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[39.93, 116.38],
    radius=30,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)



folium.CircleMarker(
    location=[39.93, 116.38],
    radius=60,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)


folium.CircleMarker(
    location=[39.93, 116.38],
    radius=90,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)


folium.CircleMarker(
    location=[39.93, 116.38],
    radius=120,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)
folium.CircleMarker(
    location=[39.93, 116.38],
    radius=150,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)
m
# m.save(os.path.join('results', 'Heatmap.html'))


In [18]:
san_map = folium.Map(location=[37.77, -122.42], zoom_start=12,width='50%',height='50%')

cdata = pd.read_csv('https://cocl.us/sanfran_crime_dataset')
#cdata = pd.read_csv('Police_Department_Incidents_-_Previous_Year__2016_.csv') #犯罪数据，包含犯罪所在经纬度

# get the first 200 crimes in the cdata
limit = 200
data = cdata.iloc[0:limit, :]
# Instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()
# Loop through the 200 crimes and add each to the incidents feature group
for lat, lng, in zip(cdata.Y, data.X):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=7, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.4
        )
    )

san_map.add_child(incidents)

In [5]:
from scipy.signal import correlate2d
from Cell2D import Cell2D, draw_array

class Schelling(Cell2D):
    """Represents a grid of Schelling agents."""
    #代表一个谢林代理的网格。

    options = dict(mode='same', boundary='wrap')

    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]], dtype=np.int8)

    def __init__(self, n, p):
        """Initializes the attributes.

        n: number of rows
        p: threshold on the fraction of similar neighbors
        初始化属性。

        n：行的数量
        p：相似邻居的比例的阈值

        """
        self.p = p
        # 0 is empty, 1 is red, 2 is blue
        choices = np.array([0, 1, 2], dtype=np.int8)
        probs = [0.1, 0.45, 0.45]
        self.array = np.random.choice(choices, (n, n), p=probs)

    def count_neighbors(self):
        """Surveys neighboring cells.

        returns: tuple of
            empty: True where cells are empty
            frac_red: fraction of red neighbors around each cell
            frac_blue: fraction of blue neighbors around each cell
            frac_same: fraction of neighbors with the same color
            调查相邻的单元格。

        Returns: tuple of
            空。如果单元格是空的，则为真
            frac_red：每个单元格周围的红色邻居的比例。
            frac_blue：每个单元格周围的蓝色邻居的比例。
            frac_same：具有相同颜色的邻居的比例。

        """
        a = self.array

        empty = a==0
        red = a==1
        blue = a==2

        # count red neighbors, blue neighbors, and total
        #计算红色邻居、蓝色邻居和总数
        num_red = correlate2d(red, self.kernel, **self.options)
        num_blue = correlate2d(blue, self.kernel, **self.options)
        num_neighbors = num_red + num_blue

        # compute fraction of similar neighbors
        #计算相似邻居的比例
        frac_red = num_red / num_neighbors
        frac_blue = num_blue / num_neighbors

        # no neighbors is considered the same as no similar neighbors
        # (this is an arbitrary choice for a rare event)
        #没有邻居被认为与没有相似的邻居相同
        #（这是对一个罕见事件的任意选择）

        frac_red[num_neighbors == 0] = 0
        frac_blue[num_neighbors == 0] = 0

        # for each cell, compute the fraction of neighbors with the same color
        #对于每个单元，计算具有相同颜色的邻居的比例
        frac_same = np.where(red, frac_red, frac_blue)

        # for empty cells, frac_same is NaN
        # 对于空单元格，frac_same为NaN
        frac_same[empty] = np.nan

        return empty, frac_red, frac_blue, frac_same

    def segregation(self):
        """Computes the average fraction of similar neighbors.

        returns: fraction of similar neighbors, averaged over cells
        计算相似邻居的平均比例。

        返回：相似邻居的比例，按单元格的平均数计算。
        """
        _, _, _, frac_same = self.count_neighbors()
        return np.nanmean(frac_same)

    def step(self):
        """Executes one time step.

        returns: fraction of similar neighbors, averaged over cells
         返回：相似邻居的比例，以单元格的平均数计算。
        """
        a = self.array
        empty, _, _, frac_same = self.count_neighbors()

        # find the unhappy cells (ignore NaN in frac_same)
        #找到不快乐的单元格（忽略frac_same中的NaN）。

        with np.errstate(invalid='ignore'):
            unhappy = frac_same < self.p
        unhappy_locs = locs_where(unhappy)

        # find the empty cells
        ## 找到空单元格
        empty_locs = locs_where(empty)

        # shuffle the unhappy cells
        #洗刷那些不开心的单元格
        if len(unhappy_locs):
            np.random.shuffle(unhappy_locs)

        # for each unhappy cell, choose a random destination
        # 对于每个不快乐的单元，选择一个随机的目的地

        num_empty = np.sum(empty)

        for source in unhappy_locs:
            i = np.random.randint(num_empty)
            dest = empty_locs[i]

            # move

            a[dest] = a[source]
            a[source] = 0
            empty_locs[i] = source

            # check that the number of empty cells is unchanged
            # 检查空单元格的数量是否保持不变
            num_empty2 = np.sum(a==0)
            assert num_empty == num_empty2

        # return the average fraction of similar neighbors
        # 返回相似邻居的平均比例
        return np.nanmean(frac_same)

    def draw(self):
        """Draws the cells."""
        return draw_array(self.array, cmap=cmap, vmax=2)

ModuleNotFoundError: No module named 'Cell2D'

In [1]:
def make_locs(n, m):
    """Makes array where each row is an index in an `n` by `m` grid.

    n: int number of rows
    m: int number of cols

    returns: NumPy array
    """
    t = [(i, j) for i in range(n) for j in range(m)]
    return np.array(t)

In [2]:
def make_visible_locs(vision):
    """Computes the kernel of visible cells.

    vision: int distance
    """
    def make_array(d):
        """Generates visible cells with increasing distance."""
        a = np.array([[-d, 0], [d, 0], [0, -d], [0, d]])
        np.random.shuffle(a)
        return a

    arrays = [make_array(d) for d in range(1, vision+1)]
    return np.vstack(arrays)

In [3]:
def distances_from(n, i, j):
    """Computes an array of distances.

    n: size of the array
    i, j: coordinates to find distance from

    returns: array of float
    计算一个数组的距离。

    n: 数组的大小
    i, j: 找出距离的座标

    返回：数组中的浮点数
    """
    X, Y = np.indices((n, n))
    return np.hypot(X - i, Y - j)


In [4]:
class Sugarscape(Cell2D):
    """Represents an Epstein-Axtell Sugarscape."""

    def __init__(self, n, **params):
        """Initializes the attributes.

        n: number of rows and columns
        params: dictionary of parameters
        """
        self.n = n
        self.params = params

        # track variables
        self.agent_count_seq = []

        # make the capacity array
        self.capacity = self.make_capacity()

        # initially all cells are at capacity
        self.array = self.capacity.copy()

        # make the agents
        self.make_agents()

    def make_capacity(self):
        """Makes the capacity array."""

        # compute the distance of each cell from the peaks.
        dist1 = distances_from(self.n, 15, 15)
        dist2 = distances_from(self.n, 35, 35)
        dist = np.minimum(dist1, dist2)

        # cells in the capacity array are set according to dist from peak
        bins = [21, 16, 11, 6]
        a = np.digitize(dist, bins)
        return a

    def make_agents(self):
        """Makes the agents."""

        # determine where the agents start and generate locations
        n, m = self.params.get('starting_box', self.array.shape)
        locs = make_locs(n, m)
        np.random.shuffle(locs)

        # make the agents
        num_agents = self.params.get('num_agents', 400)
        assert(num_agents <= len(locs))
        self.agents = [Agent(locs[i], self.params)
                       for i in range(num_agents)]

        # keep track of which cells are occupied
        self.occupied = set(agent.loc for agent in self.agents)

    def grow(self):
        """Adds sugar to all cells and caps them by capacity."""
        grow_rate = self.params.get('grow_rate', 1)
        self.array = np.minimum(self.array + grow_rate, self.capacity)

    def look_and_move(self, center, vision):
        """Finds the visible cell with the most sugar.

        center: tuple, coordinates of the center cell
        vision: int, maximum visible distance

        returns: tuple, coordinates of best cell
        """
        # find all visible cells
        locs = make_visible_locs(vision)
        locs = (locs + center) % self.n

        # convert rows of the array to tuples
        locs = [tuple(loc) for loc in locs]

        # select unoccupied cells
        empty_locs = [loc for loc in locs if loc not in self.occupied]

        # if all visible cells are occupied, stay put
        if len(empty_locs) == 0:
            return center

        # look up the sugar level in each cell
        t = [self.array[loc] for loc in empty_locs]

        # find the best one and return it
        # (in case of tie, argmax returns the first, which
        # is the closest)
        i = np.argmax(t)
        return empty_locs[i]

    def harvest(self, loc):
        """Removes and returns the sugar from `loc`.

        loc: tuple coordinates
        """
        sugar = self.array[loc]
        self.array[loc] = 0
        return sugar

    def step(self):
        """Executes one time step."""
        replace = self.params.get('replace', False)

        # loop through the agents in random order
        random_order = np.random.permutation(self.agents)
        for agent in random_order:

            # mark the current cell unoccupied
            self.occupied.remove(agent.loc)

            # execute one step
            agent.step(self)

            # if the agent is dead, remove from the list
            if agent.is_starving() or agent.is_old():
                self.agents.remove(agent)
                if replace:
                    self.add_agent()
            else:
                # otherwise mark its cell occupied
                self.occupied.add(agent.loc)

        # update the time series
        self.agent_count_seq.append(len(self.agents))

        # grow back some sugar
        self.grow()
        return len(self.agents)

    def add_agent(self):
        """Generates a new random agent.

        returns: new Agent
        """
        new_agent = Agent(self.random_loc(), self.params)
        self.agents.append(new_agent)
        self.occupied.add(new_agent.loc)
        return new_agent

    def random_loc(self):
        """Choose a random unoccupied cell.

        returns: tuple coordinates
        """
        while True:
            loc = tuple(np.random.randint(self.n, size=2))
            if loc not in self.occupied:
                return loc

    def draw(self):
        """Draws the cells."""
        draw_array(self.array, cmap='YlOrRd', vmax=9, origin='lower')

        # draw the agents
        xs, ys = self.get_coords()
        self.points = plt.plot(xs, ys, '.', color='red')[0]

    def get_coords(self):
        """Gets the coordinates of the agents.

        Transforms from (row, col) to (x, y).

        returns: tuple of sequences, (xs, ys)
        """
        agents = self.agents
        rows, cols = np.transpose([agent.loc for agent in agents])
        xs = cols + 0.5
        ys = rows + 0.5
        return xs, ys

NameError: name 'Cell2D' is not defined